In [7]:
#!/usr/bin/env python
from datetime import datetime
from time import time
from lxml import html,etree
from reviews_final import scrape, write_in_csv
import pandas as pd
import requests,re
import os,sys
import unicodecsv as csv
import argparse

def parse(locality,checkin_date,checkout_date,sort):
    checkIn = checkin_date.strftime("%Y/%m/%d")
    checkOut = checkout_date.strftime("%Y/%m/%d")
    print ("Scraper Inititated for Locality:%s"%locality)
    # TA rendering the autocomplete list using this API
    print ("Finding search result page URL")
    geo_url = 'https://www.tripadvisor.com/TypeAheadJson?action=API&startTime='+str(int(time()))+'&uiOrigin=GEOSCOPE&source=GEOSCOPE&interleaved=true&types=geo,theme_park&neighborhood_geos=true&link_type=hotel&details=true&max=12&injectNeighborhoods=true&query='+locality
    api_response  = requests.get(geo_url, verify=False).json()
    #getting the TA url for th equery from the autocomplete response
    url_from_autocomplete = "http://www.tripadvisor.com"+api_response['results'][0]['url']
    print ('URL found %s'%url_from_autocomplete)
    geo = api_response['results'][0]['value']
    #Formating date for writing to file
    a=url_from_autocomplete
    b=a.split("-")
    s="-"
    c=s.join([b[0],b[1],"oa30",b[2],b[3]])
    d=s.join([b[0],b[1],"oa60",b[2],b[3]])
    e=s.join([b[0],b[1],"oa90",b[2],b[3]])
    urllist = [a,c,d,e]

    date = checkin_date.strftime("%Y_%m_%d")+"_"+checkout_date.strftime("%Y_%m_%d")
    #form data to get the hotels list from TA for the selected date
    form_data = {'changeSet': 'TRAVEL_INFO',
            'showSnippets': 'false',
            'staydates':date,
            'uguests': '2',
            'sortOrder':sort
        
    }
    
    
    all_hotel=[]
    
    for url_from_autocomplete in urllist:
        print(url_from_autocomplete)

        headers = {
                                'Accept': 'text/javascript, text/html, application/xml, text/xml, */*',
                                'Accept-Encoding': 'gzip,deflate',
                                'Accept-Language': 'en-US,en;q=0.5',
                                'Cache-Control': 'no-cache',
                                'Connection': 'keep-alive',
                                'Content-Type': 'application/x-www-form-urlencoded; charset=utf-8',
                                'Host': 'www.tripadvisor.com',
                                'Pragma': 'no-cache',
                                'Referer': url_from_autocomplete,
                                'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:28.0) Gecko/20100101 Firefox/28.0',
                                'X-Requested-With': 'XMLHttpRequest'
                            }
        cookies=  {"SetCurrency":"USD"}
        print ("Downloading search results page")
        page_response  = requests.post(url = url_from_autocomplete,data=form_data,headers = headers, cookies = cookies, verify=False)
        print ("Parsing results ")
        parser = html.fromstring(page_response.text)
        hotel_lists = parser.xpath('//div[contains(@class,"listItem")]//div[contains(@class,"listing collapsed")]')
        hotel_data = []
        if not hotel_lists:
            hotel_lists = parser.xpath('//div[contains(@class,"listItem")]//div[@class="listing "]')

        for hotel in hotel_lists:
            XPATH_HOTEL_LINK = './/a[contains(@class,"property_title")]/@href'
            XPATH_REVIEWS  = './/a[@class="review_count"]//text()'
            XPATH_RANK = './/div[@class="popindex"]//text()'
            XPATH_RATING = './/span[contains(@class,"ui_bubble_rating bubble_45")]/@alt' #update this code to get rating
            XPATH_RATING_2 = './/a[contains(@class,"ui_bubble_rating bubble_45")]/@alt' #update this code to get rating
            XPATH_HOTEL_NAME = './/a[contains(@class,"property_title")]//text()'
            XPATH_HOTEL_FEATURES = './/div[contains(@casls,"common_hotel_icons_list")]//li//text()'
            XPATH_HOTEL_PRICE = './/div[contains(@data-sizegroup,"mini-meta-price")]/text()'
            XPATH_VIEW_DEALS = './/div[contains(@data-ajax-preserve,"viewDeals")]//text()'
            XPATH_BOOKING_PROVIDER = './/div[contains(@data-sizegroup,"mini-meta-provider")]//text()'

            raw_booking_provider = hotel.xpath(XPATH_BOOKING_PROVIDER)
            raw_no_of_deals =  hotel.xpath(XPATH_VIEW_DEALS)
            raw_hotel_link = hotel.xpath(XPATH_HOTEL_LINK)
            raw_no_of_reviews = hotel.xpath(XPATH_REVIEWS)
            raw_rank = hotel.xpath(XPATH_RANK)
            raw_rating = hotel.xpath(XPATH_RATING_2)
            raw_hotel_name = hotel.xpath(XPATH_HOTEL_NAME)
            raw_hotel_features = hotel.xpath(XPATH_HOTEL_FEATURES)
            raw_hotel_price_per_night  = hotel.xpath(XPATH_HOTEL_PRICE)

            url = 'http://www.tripadvisor.com'+raw_hotel_link[0] if raw_hotel_link else  None
            reviews = ''.join(raw_no_of_reviews).replace("reviews","").replace(",","") if raw_no_of_reviews else 0
            rank = ''.join(raw_rank) if raw_rank else None
            rating = ''.join(raw_rating).replace('of 5 bubbles','').strip() if raw_rating else None
            name = ''.join(raw_hotel_name).strip() if raw_hotel_name else None
            hotel_features = ','.join(raw_hotel_features)
            #price_per_night = ''.join(raw_hotel_price_per_night).encode('utf-8').replace('\n','') if raw_hotel_price_per_night else None
            price_per_night = ''.join(raw_hotel_price_per_night).replace('\n','') if raw_hotel_price_per_night else None

            no_of_deals = re.findall("all\s+?(\d+)\s+?",''.join(raw_no_of_deals))
            booking_provider = ''.join(raw_booking_provider).strip() if raw_booking_provider else None

            if no_of_deals:
                no_of_deals = no_of_deals[0]
            else:
                no_of_deals = 0

            data = {
                        'hotel_name':name,
                        'url':url,
                        'locality':locality,
                        'reviews':reviews,
                        'rank':rank,
                        'tripadvisor_rating':rating,
                        'checkOut':checkOut,
                        'checkIn':checkIn,
                        'hotel_features':hotel_features,
                        'price_per_night':price_per_night,
                        'no_of_deals':no_of_deals,
                        'booking_provider':booking_provider

            }
            hotel_data.append(data)
            all_hotel.append(data)
    #Referrer is necessary to get the correct response from TA if not provided they will redirect to home page
    my_df=pd.DataFrame(all_hotel)
    print(my_df['hotel_name'])
    return all_hotel
   


In [8]:
test="2021/11/01,2021/11/05,popularity,Milan"

In [11]:
parser = argparse.ArgumentParser()
parser.add_argument('checkin_date',help = 'Hotel Check In Date (Format: YYYY/MM/DD')
sortorder_help = """
    available sort orders are :\n
    priceLow - hotels with lowest price,
    distLow : Hotels located near to the search center,
    recommended: highest rated hotels based on traveler reviews,
    popularity :Most popular hotels as chosen by Tipadvisor users
    """
parser.add_argument('checkout_date',help = 'Hotel Chek Out Date (Format: YYYY/MM/DD)')
parser.add_argument('sort',help = sortorder_help,default ='popularity ')
parser.add_argument('locality',help = 'Search Locality')
print(test)
args = parser.parse_args(test.split(','))
print (args)
locality = args.locality
checkin_date = datetime.strptime(args.checkin_date,"%Y/%m/%d")
checkout_date = datetime.strptime(args.checkout_date,"%Y/%m/%d")
sort= args.sort
checkIn = checkin_date.strftime("%Y/%m/%d")
checkOut = checkout_date.strftime("%Y/%m/%d")
today = datetime.now()

2021/11/01,2021/11/05,popularity,Milan
Namespace(checkin_date='2021/11/01', checkout_date='2021/11/05', sort='popularity', locality='Milan')


In [12]:
checkIn = checkin_date.strftime("%Y/%m/%d")
checkOut = checkout_date.strftime("%Y/%m/%d")
print ("Scraper Inititated for Locality:%s"%locality)
# TA rendering the autocomplete list using this API


Scraper Inititated for Locality:Milan


In [27]:
print ("Finding search result page URL")
geo_url = 'https://www.tripadvisor.com/TypeAheadJson?action=API&startTime='+str(int(time()))+'&uiOrigin=GEOSCOPE&source=GEOSCOPE&interleaved=true&types=geo,theme_park&neighborhood_geos=true&link_type=hotel&details=true&max=12&injectNeighborhoods=true&query='+locality
api_response  = requests.get(geo_url, timeout=10).json()
#getting the TA url for th equery from the autocomplete response
url_from_autocomplete = "http://www.tripadvisor.com"+api_response['results'][0]['url']
print ('URL found %s'%url_from_autocomplete)
geo = api_response['results'][0]['value']
#Formating date for writing to file
a=url_from_autocomplete
b=a.split("-")
s="-"
c=s.join([b[0],b[1],"oa30",b[2],b[3]])
d=s.join([b[0],b[1],"oa60",b[2],b[3]])
e=s.join([b[0],b[1],"oa90",b[2],b[3]])
urllist = [a,c,d,e]

date = checkin_date.strftime("%Y_%m_%d")+"_"+checkout_date.strftime("%Y_%m_%d")
#form data to get the hotels list from TA for the selected date
form_data = {'changeSet': 'TRAVEL_INFO',
        'showSnippets': 'false',
        'staydates':date,
        'uguests': '2',
        'sortOrder':sort
    
}

Finding search result page URL
URL found http://www.tripadvisor.com/Hotels-g187849-Milan_Lombardy-Hotels.html


In [14]:
geo_url

'https://www.tripadvisor.com/TypeAheadJson?action=API&startTime=1635057042&uiOrigin=GEOSCOPE&source=GEOSCOPE&interleaved=true&types=geo,theme_park&neighborhood_geos=true&link_type=hotel&details=true&max=12&injectNeighborhoods=true&query=Milan'

In [22]:
s = 'https://www.tripadvisor.com/TypeAheadJson?action=API&startTime=1635057042&uiOrigin=GEOSCOPE&source=GEOSCOPE&interleaved=true&types=geo,theme_park&neighborhood_geos=true&link_type=hotel&details=true&max=12&injectNeighborhoods=true&query=Milan'

In [23]:
geo_url == s

True

In [26]:
api_response

{'normalized': {'query': 'milan'},
 'query': {'link_type': 'hotel',
  'types': 'geo,theme_park',
  'neighborhood_geos': 'true',
  'uiOrigin': 'GEOSCOPE',
  'max': '12',
  'query': 'Milan',
  'interleaved': 'true',
  'startTime': '1635057042',
  'details': 'true',
  'source': 'GEOSCOPE'},
 'results': [{'lookbackServlet': None,
   'autobroadened': 'false',
   'title': 'Destinations',
   'type': 'GEO',
   'document_id': '187849',
   'url': '/Hotels-g187849-Milan_Lombardy-Hotels.html',
   'urls': [{'url_type': 'hotel',
     'name': 'Milan Hotels',
     'fallback_url': '/Hotels-g187849-Milan_Lombardy-Hotels.html',
     'type': 'HOTEL',
     'url': '/Hotels-g187849-Milan_Lombardy-Hotels.html'}],
   'scope': 'global',
   'name': 'Milan, Lombardy, Italy',
   'data_type': 'LOCATION',
   'details': {'placetype': 10004,
    'parent_name': 'Italy',
    'grandparent_name': 'Europe',
    'grandparent_id': 4,
    'parent_id': 187768,
    'grandparent_place_type': 10000,
    'rac_enabled': True,
    '

In [25]:
api_response  = requests.get(geo_url, timeout=10).json()

In [ ]:
len(data)

In [15]:
my_df = pd.DataFrame(data)

In [16]:
my_df.head(15)

,hotel_name,url,locality,reviews,rank,tripadvisor_rating,checkOut,checkIn,hotel_features,price_per_night,no_of_deals,booking_provider
0,NYX Hotel Milan,http://www.tripadvisor.com/Hotel_Review-g18784...,Milan,854,"#1 Best Value of 1,292 places to stay in Milan",4.5,2020/11/05,2020/11/01,,$126,19,Leonardo
1,Hilton Garden Inn Milan North,http://www.tripadvisor.com/Hotel_Review-g18784...,Milan,1362,"#2 Best Value of 1,292 places to stay in Milan",None,2020/11/05,2020/11/01,,$119,13,Booking.com
2,Hotel Berna,http://www.tripadvisor.com/Hotel_Review-g18784...,Milan,7709,"#3 Best Value of 1,292 places to stay in Milan",4.5,2020/11/05,2020/11/01,,$200,13,eDreams
3,ibis Milano Centro,http://www.tripadvisor.com/Hotel_Review-g18784...,Milan,7709,"#4 Best Value of 1,292 places to stay in Milan",None,2020/11/05,2020/11/01,,$79,13,ibis
4,The Square Milano Duomo,http://www.tripadvisor.com/Hotel_Review-g18784...,Milan,1102,"#5 Best Value of 1,292 places to stay in Milan",4.5,2020/11/05,2020/11/01,,$237,20,Booking.com
5,Enterprise Hotel,http://www.tripadvisor.com/Hotel_Review-g18784...,Milan,2037,"#6 Best Value of 1,292 places to stay in Milan",4.5,2020/11/05,2020/11/01,,$107,11,Booking.com
6,Klima Hotel Milano Fiere,http://www.tripadvisor.com/Hotel_Review-g18784...,Milan,1667,"#7 Best Value of 1,292 places to stay in Milan",None,2020/11/05,2020/11/01,,$99,12,Agoda.com
7,Chateau Monfort,http://www.tripadvisor.com/Hotel_Review-g18784...,Milan,2053,"#8 Best Value of 1,292 places to stay in Milan",4.5,2020/11/05,2020/11/01,,$208,14,Booking.com
8,Hyatt Centric Milan Centrale,http://www.tripadvisor.com/Hotel_Review-g18784...,Milan,1583,"#9 Best Value of 1,292 places to stay in Milan",None,2020/11/05,2020/11/01,,$186,18,eDreams
9,Michelangelo Hotel,http://www.tripadvisor.com/Hotel_Review-g18784...,Milan,2042,"#10 Best Value of 1,292 places to stay in Milan",None,2020/11/05,2020/11/01,,$148,13,eDreams


In [ ]:
print(my_df['hotel_name'])

In [ ]:
for i in data:
    for y in i:
        print (y,':',i[y])

In [ ]:
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Paris"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Chicago"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "New York"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "San Francisco"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Istanbul"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Singapore"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Bombay"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Bangkok"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Sydney"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "London"

In [ ]:
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Rome"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Athens"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Miami"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Austin"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Tokyo"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Bali"

In [ ]:
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Osaka"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Hong Kong"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Dubai"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Phuket"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Seoul"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Prague"
%run trip_advisor_final.py "2019/10/01" "2019/10/05" "popularity" "Milan"

In [ ]:
test="2020/11/01,2020/11/05,popularity,Milan"

In [ ]:
test.split(',')

In [ ]:
args
print(locality)

In [ ]:
if today<datetime.strptime(checkIn,"%Y/%m/%d") and datetime.strptime(checkIn,"%Y/%m/%d")<datetime.strptime(checkOut,"%Y/%m/%d"):
        data = parse(locality,checkin_date,checkout_date,sort)
        print ("Writing to output file tripadvisor_data.csv")
        with open('tripadvisor_data.csv','wb')as csvfile:
            fieldnames = ['hotel_name','url','locality','reviews','tripadvisor_rating','checkIn','checkOut','price_per_night','booking_provider','no_of_deals','hotel_features']
            my_df  = pd.DataFrame(columns = fieldnames)
            for row in  data:
                my_df.loc[len(my_df)] = [row['hotel_name'],row['url'],row['locality'],row['reviews'],row['tripadvisor_rating'],row['checkIn'],row['checkOut'],row['price_per_night'],row['booking_provider'],row['no_of_deals'],row['hotel_features']]
            DB_COLUMN   = 'review_body'
            DB_COLUMN1 = 'review_date'
            DB_COLUMN2 = 'hotelName'
            DB_COLUMN3 = 'hotelUrl'
            start_urls = []
            for index, row2 in my_df.iterrows():
                start_urls.append(row2['url'])
            lang = 'en'
    
            headers = [
            DB_COLUMN,
            DB_COLUMN1,
            DB_COLUMN2,
            DB_COLUMN3
            ]
            reviews_df  = pd.DataFrame(columns = headers)
            
            for url in start_urls:

                # get all reviews for 'url' and 'lang'
                items = scrape(url, lang)
                filename = 'hotelReviewsIn' + locality + '__' + lang
                filename2 = "HotelListIn" + locality + '__' + lang
                if not items:
                    print('No reviews')
                else:
                    # write in CSV
                    for i in items:
                        reviews_df.loc[len(reviews_df)] = [i['review_body'],i['review_date'],my_df.loc[my_df['url'] == url]['hotel_name'], url]
            ci = checkIn.replace("/", "")
            co = checkOut.replace("/", "")
            reviews_df.to_csv(filename + ci +co+'.csv', encoding='utf-8')     
            my_df.to_csv(filename2 + ci +co+'.csv', encoding='utf-8')       
    #checking whether the entered date is already passed
elif today>datetime.strptime(checkIn,"%Y/%m/%d") or today>datetime.strptime(checkOut,"%Y/%m/%d"):
     print ("Invalid Checkin date: Please enter a valid checkin and checkout dates,entered date is already passed")

elif datetime.strptime(checkIn,"%Y/%m/%d")>datetime.strptime(checkOut,"%Y/%m/%d"):
     print ("Invalid Checkin date: CheckIn date must be less than checkOut date")


In [6]:
data = parse(locality,checkin_date,checkout_date,sort)

NameError: name 'locality' is not defined

In [ ]:
len(data)

In [ ]:
http://www.tripadvisor.com/Hotels-g187849-Milan_Lombardy-Hotels.html
https://www.tripadvisor.com/Hotels-g187849-oa30-Milan_Lombardy-Hotels.html
https://www.tripadvisor.com/Hotels-g187849-oa60-Milan_Lombardy-Hotels.html

In [ ]:
a="http://www.tripadvisor.com/Hotels-g187849-Milan_Lombardy-Hotels.html"

In [ ]:
b=a.split("-")

In [ ]:
print (b)

In [ ]:
print(b[0])

In [ ]:
s="-"

In [ ]:
c=s.join([b[0],b[1],"oa30",b[2],b[3]])
d=s.join([b[0],b[1],"oa60",b[2],b[3]])

In [ ]:
urllist = [a,c,d]

In [ ]:
urllist